In [1]:
!pip install textblob
!pip install textacy
!pip install pyspellchecker
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 12.5 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 20.2 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.4/208.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:0031m29.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 17.3 MB/s eta 0:00:00m eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.2 MB/s eta 0:00:00m eta 0:00:0136m0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 11.9 MB/s eta 0:00:0031m16.4 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=9f85a81085b4b79d693c4218d64dd8ee8750cb968ad91850d62afc1e2219ed89
  Stored in directory: /home/hfsc5/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect


In [1]:
import pandas as pd
import numpy as np
import string
import re
import textacy

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

In [2]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
with open("combined.txt", "rb") as f:
    data = f.readlines()

In [6]:
data = [item.strip() for item in data]

In [7]:
df = pd.DataFrame({'chats': data})
df['chats'] = df['chats'].astype("string")

In [8]:
df['users'] = df['chats'].str.split(n=1).str[0]
df['chats'] = df['chats'].str.split(n=1).str[1]

In [9]:
df['chats'] = df['chats'].str.replace('\d+', '')#remove numbers
df['chats']=df['chats'].str.replace(r"[\/\-\~\_\[\]\{\}\:\;\(\)\"\'\|\?\=\.\<\>\@\#\*\,]", '')#remove punctuations
df['chats'] = df['chats'].str.replace('[^\w\s]','')#remove punctuations

/tmp/ipykernel_18265/1211096447.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('\d+', '')#remove numbers
/tmp/ipykernel_18265/1211096447.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats']=df['chats'].str.replace(r"[\/\-\~\_\[\]\{\}\:\;\(\)\"\'\|\?\=\.\<\>\@\#\*\,]", '')#remove punctuations
/tmp/ipykernel_18265/1211096447.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('[^\w\s]','')#remove punctuations


In [10]:
df = df.dropna()
exploded_series = df['chats'].str.split(' ').explode()
mask = exploded_series.isin(['PM','AM'])
df['time'] = exploded_series[mask].groupby(level=0).apply(' '.join)
df['chats'] = exploded_series[~mask].groupby(level=0).apply(' '.join)
df = df.fillna('')

In [11]:
df["Number of Words"] = df["chats"].apply(lambda n: len(n.split()))

In [12]:
df.sort_values(by=['Number of Words'], ascending=[False])

,chats,users,time,Number of Words
46841,the next phone call which lasted approximately...,b'In,PM,330
474981,hi love i have look see if u came back on a...,b'ghost27_73,PM,177
119164,You are beautiful Trust me on this You are t...,b'atlanta.italian,PM,160
119163,When you are gone I have time to think You a...,b'atlanta.italian,PM,160
120062,Maddie I wanted to drop you a note saying ho...,b'tealitalianeyes,PM,157
...,...,...,...,...
883321,,b'malhotra.mohit,PM,0
883322,,b'malhotra.mohit,PM,0
300947,,b'makenna_againa,PM,0
300922,,b'makenna_againa,PM,0


In [13]:
df1=df.groupby(['users'])['chats'].apply(','.join).reset_index()

In [14]:
df1["Number of Words"] = df["chats"].apply(lambda n: len(n.split()))

In [15]:
corpus = df['chats'].tolist()
len(corpus)

1352907

In [16]:
corpus2 = df1['chats'].tolist()
len(corpus)

1352907

In [17]:
corpus = list(dict.fromkeys(corpus))
#Remove words with less than 2 characters
corpus = [re.sub(r'\b\w{0,1}\b', '', d) for d in corpus]
corpus = list(dict.fromkeys(corpus))
corpus = list(filter(None, corpus))
corpus = [c.strip(' ') for c in corpus]
corpus = [c.lower() for c in corpus]

In [18]:
def check_long(lst):
    return [item for item in lst if len(item) > 10]

In [19]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /home/hfsc5/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [20]:
from nltk.corpus import brown

In [21]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/hfsc5/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
from textblob import Word

In [23]:
corpus_2 = [re.sub(r'\b\w{0,1}\b', '', d) for d in corpus]
corpus_2 = list(dict.fromkeys(corpus_2))
corpus_2 = list(filter(None, corpus_2))
corpus_2 = [c.strip(' ') for c in corpus_2]

In [24]:
in_words =[]
for line in corpus_2:
    for word in line.split():
        if (word.endswith('in')) and (word not in in_words):
            in_words.append(word)

In [25]:
word_list = brown.words()
word_list = [wl.lower() for wl in word_list]

In [26]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
wnl = WordNetLemmatizer()
word_lem=[]
to_change_in=[]

for w in in_words:
    if (wnl.lemmatize(''.join([w,'g']), pos='v') in word_list) and (w!='thin'):
        to_change_in.append(w)
        word_lem.append(wnl.lemmatize(''.join([w,'g']), pos='v'))

In [27]:
def get_tuple(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [28]:
tuple_pair_in = get_tuple(to_change_in, word_lem)

In [29]:
# tuple_pair_in[:-20]

In [30]:
temp_str = str(corpus_2)

In [31]:
for old, new in tuple_pair_in:
    temp_str = temp_str.replace(old, new)

In [32]:
temp_str=temp_str.replace('gg','g')

In [33]:
fix_words = ['suked', 'fuked', 'suk', 'fuk', 'suking', 'fuking']
fixed_words = ['sucked', 'fucked','suk', 'fuck', 'sucking', 'fucking']
fix_tuple = get_tuple(fix_words, fixed_words)

In [34]:
fixed_words = ['sucked', 'fucked','suk', 'fuck', 'sucking', 'fucking', 'porn']

In [35]:
for old, new in fix_tuple:
    temp_str = temp_str.replace(old, new)

In [36]:
chats_1 = temp_str.split(',')
chats_1 = [re.sub(r'\b\w{0,1}\b', '', d) for d in chats_1]
chats_1 = list(dict.fromkeys(chats_1))
chats_1 = list(filter(None, chats_1))
chats_1 = [c.strip(' ') for c in chats_1]

In [37]:
import textacy 

patterns = [{'POS':'X'}]

other_words2 = []

for chat in chats_1:
    doc= nlp(chat)
    other_text = textacy.extract.token_matches(doc, patterns = patterns)
    for o in other_text:
        rmv_txt = str(o)
        if (rmv_txt not in other_words2):
            other_words2.append(rmv_txt)

In [38]:
df = pd.DataFrame({'chats': chats_1})
df['chats'] = df['chats'].astype("string")
df['chats'] = df['chats'].str.replace('[^\w\s]','')

/tmp/ipykernel_18265/1749365649.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('[^\w\s]','')


In [39]:
data = df.chats.tolist()
data = list(filter(None, data))
data = list(dict.fromkeys(data))

In [40]:
data_str = str(data)

In [41]:
with open('short_words.txt', 'r') as f:
    short_words = f.readlines()

In [42]:
short_words = [re.sub('\s+', ' ', s) for s in short_words]
short_words = [re.sub('\d+', ' ', s) for s in short_words]

In [43]:
short_words2 = []
for sw in short_words:
    short_words2.append(' '+sw)

In [44]:
for sw in short_words2:
    data_str=data_str.replace(sw, ' ')

In [45]:
data_2 = data_str.split(',')

In [46]:
lol_ptrn = 'lol\w+'
lol_words = re.findall(lol_ptrn, data_str)
lol_words = list(dict.fromkeys(lol_words))

In [47]:
lol_ptrn = 'lol\w+'
lol_words = re.findall(lol_ptrn, data_str)
lol_words = list(dict.fromkeys(lol_words))

In [48]:
lol_words2=[]
for line in data_2:
    for word in line.split():
        if ('lol' in word) and (word not in lol_words2):
            lol_words2.append(word)

In [49]:
for i in lol_words:
    data_str = data_str.replace(i,'')

In [50]:
for i in lol_words2:
    data_str = data_str.replace(i,'')

In [51]:
ok_ptrn = 'ok\w+'
ok_words = re.findall(ok_ptrn, data_str)
ok_words = list(dict.fromkeys(ok_words))

In [52]:
for i in ok_words:
    data_str = data_str.replace(i,'')

In [53]:
ing_ptrn = 'ing\w+'
ing_words = re.findall(ing_ptrn, data_str)
ing_words = list(dict.fromkeys(ing_words))

In [54]:
ing_words2=[]
for i in ing_words:
    ing_words2.append(' '+i)

In [55]:
# cocked_ptrn = 'cocked\w+'
# cocked_words = re.findall(cocked_ptrn, data_str)
# cocked_words = list(dict.fromkeys(cocked_words))

In [56]:
for i in ing_words2:
    data_str = data_str.replace(i,'')

In [57]:
# cocked_words2=[]
# for c in cocked_words:
#     cocked_words2.append(' '+c)

In [58]:
# def add_ws(string, length):
#     return ' '.join(string[i:i+length] for i in range(0,len(string),length))

In [59]:
# cocked_words2=[]

# for c in cocked_words:
#     cocked_words2.append(add_ws(c, 6))

In [60]:
# tuple_cocked = get_tuple(cocked_words, cocked_words2)

In [61]:
# for old, new in tuple_cocked:
#     corpus_str = corpus_str.replace(old,new)

In [62]:
chats = data_str.split(',')

In [63]:
#Remove words with less than 2 characters
chats = [re.sub(r'\b\w{0,1}\b', '', d) for d in chats]
chats = list(dict.fromkeys(chats))
chats = list(filter(None, chats))
chats = [c.strip(' ') for c in chats]
chats = [c.lower() for c in chats]

In [64]:
df = pd.DataFrame({'chats': chats})

In [65]:
df['chats'] = df['chats'].astype("string")
df['chats'] = df['chats'].str.replace('[^\w\s]','')

/tmp/ipykernel_18265/3449276781.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('[^\w\s]','')


In [66]:
df['chats']=df['chats'].str.lower()

In [67]:
df['chats'] = df['chats'].apply(lambda x: re.split('http\/\/.*', str(x))[0])
df['chats'] = df['chats'].apply(lambda x: re.split('https\/\/.*', str(x))[0])
df['chats'] = df['chats'].apply(lambda x: re.split('www\/\/.*', str(x))[0])

In [68]:
def remove_whitespaces(text):
    rmv_ws = re.sub(' +', ' ' ,text)
    return rmv_ws

def remove_hl(text):
    final = re.sub('<[a][^>]*>(.+?)</[a]>', 'Link.', text)
    return final

In [69]:
def cleaning(x, chat):
    df_x = x.copy()
    df_x[chat]= df_x['chats']
    df_x[chat]= df_x[chat].apply(remove_whitespaces)
    df_x[chat]= df_x[chat].apply(remove_hl)
    df_x[chat] = df_x[chat].dropna()
    return df_x

df_x = cleaning(df, 'chats')

In [70]:
corpus = df_x.chats.tolist()
corpus = list(dict.fromkeys(corpus))
corpus = list(filter(None, corpus))
print(len(corpus))

740322


In [71]:
to_change=[]
changed = []
for line in corpus:
    for word in line.split():
        if len(word)>=13:
            to_change.append(word)
            changed.append(re.sub(r'(.)\1+', r'\1', word))

In [72]:
tuple_correct = get_tuple(to_change, changed)

In [73]:
corpus_str = str(corpus)

In [74]:
for old, new in tuple_correct:
    corpus_str = corpus_str.replace(old, new)

In [75]:
corpus_2 = corpus_str.split(',')

In [76]:
corpus_2[:20]

["['hi'",
 " 'pftloser'",
 " 'sup'",
 " 'location'",
 " 'hello'",
 " 'srry was sleep'",
 " 'farmington hills'",
 " 'canton'",
 " 'do you have picture'",
 " 'yea trade'",
 " 'mine is in my profile'",
 " 'profis not work me can send'",
 " 'please send me yours will reply'",
 " 'kk sent'",
 " 'handsome boy'",
 " 'am years old'",
 " 'im'",
 " 'send'",
 " 'wow'",
 " 'dont luk that old'"]

In [77]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hfsc5/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [78]:
#Remove words with less than 2 characters
corpus_2 = [re.sub(r'\b\w{0,1}\b', '', d) for d in corpus_2]
corpus_2 = list(dict.fromkeys(corpus_2))
corpus_2 = list(filter(None, corpus_2))
corpus_2 = [c.strip(' ') for c in corpus_2]
corpus_2 = [c.lower() for c in corpus_2]

In [79]:
unique = []
for line in corpus_2:
    for word in line.split(' '):
        if word not in unique:
            unique.append(word)

In [80]:
correct_words = []
incorrect_words = []
def spell_checker(word):
    word = Word(word)
    final = word.spellcheck()
    if (wnl.lemmatize(final[0][0], pos='v')):
        if (final[0][1] > 0.56) and (final[0][1] < 0.9) and (word not in fixed_words) and (word not in word_list):
            incorrect_words.append(word)
            correct_words.append(final[0][0])
            # print(f'|"{word}": "{final[0][0]}"  | Confidence-> {final[0][1]}.')

In [81]:
for word in unique:
    spell_checker(word)

In [82]:
incorrect_w2 = []
for word in incorrect_words:
    incorrect_w2.append(' '+word+' ')

In [83]:
correct_w2 = []
for word in correct_words:
    correct_w2.append(' '+word+' ')

In [84]:
incorrect_w3=[]

In [85]:
incorrect_w3 = [re.sub(r'[^\w\s]','', d) for d in incorrect_w2]
correct_w2 = [re.sub(r'[^\w\s]','', d) for d in correct_w2]

In [86]:
tuple_correct_1 = get_tuple(incorrect_w3, correct_w2)

In [87]:
corpus_str = str(corpus_2)

In [88]:
len(tuple_correct_1)

16141

In [89]:
for old, new in tuple_correct_1:
    corpus_str = corpus_str.replace(old, new)

In [90]:
temp = corpus_str.split(',')

In [91]:
temp = [re.sub(r'\b\w{0,1}\b', '', d) for d in temp]
temp = list(dict.fromkeys(temp))
temp = list(filter(None, temp))
temp = [c.strip(' ') for c in temp]
temp = [c.lower() for c in temp]

In [92]:
df=[]

In [93]:
df = pd.DataFrame({'chats': temp})
df['chats'] = df['chats'].astype("string")
df['chats'] = df['chats'].str.replace('[^\w\s]','')

/tmp/ipykernel_18265/2229609577.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('[^\w\s]','')


In [94]:
temp_2 = df.chats.tolist()

In [95]:
temp_2 = [re.sub(r'\b\w{0,1}\b', '', d) for d in temp_2]
temp_2 = list(dict.fromkeys(temp_2))
temp_2 = list(filter(None, temp_2))
temp_2 = [c.strip(' ') for c in temp_2]
temp_2 = [c.lower() for c in temp_2]

In [96]:
# import textacy 

# patterns = [{'POS':'X'}]

# other_words = []

# for chat in temp:
#     doc= nlp(chat)
#     other_text = textacy.extract.token_matches(doc, patterns = patterns)
#     for o in other_text:
#         rmv_txt = str(o)
#         if (rmv_txt not in other_words):
#             other_words.append(rmv_txt)

In [97]:
keep_words=['heaven','transformer','risingg','snowy','drinkingg','lastnite','feb','rue','rubbingg','talkingg','makingg','versus','anythingg','christmas','english','girlish','afterschol','condemn','jokingg','anyproblem','africa','penis','caucasian','thighs','jewel','spagheti']

In [98]:
# other_words

In [99]:
# other_w = [word for word in other_words if word not in keep_words]

In [100]:
chat_str = str(temp_2)

In [101]:
str_2=re.findall("[a-zA-Z,.]+",chat_str)
chat_docx=(" ".join(str_2))

In [102]:
# for o in other_w:
#     chat_docx2=chat_docx.replace(o, '')

In [103]:
# chat_docx

In [104]:
chat_list = chat_docx.split(',')

In [105]:
pp = []
for line in chat_list:
    for word in line.split():
        if (len(word)>=12) and (word not in pp) and (word not in word_list):
            pp.append(word)

In [106]:
xx = []
for x in pp:
    xx.append((re.sub(r'(.)\1+', r'\1', x)))

In [107]:
tuple_xp = get_tuple(pp, xx)

In [108]:
# tuple_xp[:20]

In [109]:
for old, new in tuple_xp:
    chat_docx = chat_docx.replace(old, new)

In [110]:
chat_list = chat_docx.split(',')

In [111]:
chat_docx=chat_docx.replace(' ill ','')

In [112]:
chat_list = chat_docx.split(',')

In [113]:
# chat_list[:50]

In [114]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True, min_len=1, max_len=20))  # deacc=True removes punctuations

data_words = list(sent_to_words(chat_list))

In [115]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [116]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/hfsc5/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [117]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [118]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

In [119]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [120]:
df=[]
df = pd.DataFrame({'chats': data_lemmatized})
df['chats'] = df['chats'].astype("string")
df['chats'] = df['chats'].str.replace('[^\w\s]','')

/tmp/ipykernel_18265/3260702434.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['chats'] = df['chats'].str.replace('[^\w\s]','')


In [121]:
final_df = df.chats.tolist()

In [122]:
final_df = [re.sub(r'\b\w{0,1}\b', '', d) for d in final_df]
final_df = list(dict.fromkeys(final_df))
final_df = list(filter(None, final_df))
final_df = [c.strip(' ') for c in final_df]

In [123]:
# corpus[:30]

In [124]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

In [125]:
# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [126]:
# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [127]:
with open("final_corpus.txt", "w") as f:
    for text in final_df:
        f.write(text + "\n")